# **GEOG 60.01** ***Problem Set #2***

### Instructions

- **Save a copy of this PS to your jhub directory**, naming the file as follows:
`lastname_firstname_pset2_geog60.ipynb`

- **Complete the problems below by typing answers in additional cells you have created**. Save often.

- Some assignments have **optional bonus problems**. These are meant to be interesting and thought-provoking, but are not required. Extra credit will be given for interesting answers to the bonus problems.

- Remember to set your cell types to `Markdown` for text, and `Code` for Python code!

- **Include comments** in any code to explain your method as necessary. We want to give you points, not take them away.

- Remember to actually answer the questions. **Typed responses to questions in cells you create are required** (not just code and figures!)

- You can work on the problem set in groups, with two caveats: (1) **everyone must turn in their own copy in their own words**; (2) **you must list the names of your collaborators in a cell at the end of the problem set**

- Your solutions should all be in this **single Jupyter notebook** that contains your text, your code, and your figures.

- **To submit this PS once you're done, do the following:**
    1. *Make sure that your notebook* ***runs cleanly without errors:***
        - Save your notebook
        - From the `Kernel` menu, select `Restart & Run All`
        - Did the notebook run from start to finish without error and produce the expected output?
        - If yes, save again and proceed to the next step
        - If no, fix the errors and try again
    2. Download your saved PS to your computer with all your answers and figures, `code` and `Markdown` rendered. 
    3. Turn in your PS by uploading it to Canvas under the PS#2 assigment.

### Learning goals
This problem set is intended to apply and extend concepts from class, whether around modeling practices and pitfalls generally, or Earth System Modeling specifically.

Students completing this assignment will gain the following skills and concepts:

- More familiarity with [FaIR](https://docs.fairmodel.net/en/latest/intro.html), our simple climate model emulator
- An introduction to climate change scenarios that are inputs to Earth system models
- An introduction to the idea of "ensembles" of climate or Earth system models
- An introduction to the sources of uncertainty in predictions of climate change
- Thinking about high dimensional data (e.g., dims=[lon,lat,time,model,experiment,realization])
- Deeper familiarity with scientific figures and data interpretation
- Some additional programming skills, particularly with plotting in Xarray

Those are our ambitions. Ok, onward...let's load some libraries to use.

In [ ]:
# import python modules
import os
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.colors as colors

In [ ]:
# import FaIR version 2.1.0
#https://docs.fairmodel.net/en/latest/intro.html

import fair
from fair import FAIR
from fair.io import read_properties
from fair.interface import fill, initialise

---
# Problem 1. The Shared Socioeconomic Pathways (SSPs)
---

Right, so we don't know what the future holds. 

When it comes to predicting climate change, this is a problem.

Consider the fact that we don't know what technology will emerge in the coming decades, what policies will be in place, what demographic changes will occur, geopolitical and socio-economic factors will become determinant, and how all of those things together will conspire to shape fossil fuel consumption. It is, as we say, uncertain: many outcomes are consistent with expectations.

So rather than **predict** the future, climate scientists work to **project** the future. 

We make "projections," not "predictions". This is not pedantic. The distinction matters. ***A prediction is a forecast.*** Weather models make predictions. It is a statement about the future based on our best knowledge of the present. 

In contrast, ***a projection is a prediction that is "conditioned" on a set of assumptions.*** 

You can think of it as an `IF->THEN`, type of prediction. Here we scientists condition our prediction of future on assumptions about greenhouse gas (GHG) concentrations in our atmosphere.

So rather than predict what our socio-economic and political structures will be, scientists **sample** from the range of plausible possibilities: a world with high concentrations of GHGs, low GHGs, and a bunch of possibilities in between, for example. To be clear, there are no likelihoods attached to any of these concentration pathways. They're simply possibilities. **They are not policy recommendations.**

In climate parlance, these time-dependent trajectories of future GHG concentrations are called [**Representative Concentration Pathways**](https://doi.org/10.1007/s10584-011-0148-z). These pathways are all called "RCPs" and the number that follows, 8.5, 7, 4.5, etc., refers to the approximate top of the atmosphere (TOA) radiative imbalance (or forcing) at 2100 due to the concentration trajectory, i.e., $\Delta(\text{IN} - \text{OUT})$. So RCP8.5 has about 8.5 W m$^{-2}$ radiative imbalance at 2100. Recall our linearization of the forcing-response relationship from class, which can be written as:

$$\Delta F =\lambda\Delta T$$

Higher $\Delta F$ is going to mean a larger $\Delta T$: more forcing equals more warming. You plug up more of the drain in our bathtub, the water level has to rise more to reach equilibrium, such that $\text{IN} = \text{OUT}$.

<center> <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Atmospheric_CO%E2%82%82_concentrations_by_SSP_across_the_21st_century.svg/2880px-Atmospheric_CO%E2%82%82_concentrations_by_SSP_across_the_21st_century.svg.png' width="600"></center>

From: Wikipedia

The RCPs are physical boundary conditions for the climate model: at each time step, the atmosphere has this many parts per million (ppm) of CO$_2$ or that many parts per billion (ppb) CH$_4$. Implicitly, these choices suggest something about our society and its development, but not explicitly.

One can imagine, for example, that there are many ways our society could arrive at the same level of radiative forcing. Perhaps a high forcing world like RCP8.5, for instance, could only occur becuase everyone relies exclusively on fossil fuels and no countries cooperate on climate change. 

So to make these societal possibilities explicit, scientists developed a **scenario framework** to generate societal narratives that are consistent with the physics-based RCPs. To do so, they used another class of model that couples the climate and economy together (usually very simplistically) called [**Integrated Assessment Models (IAMs)**](https://www.carbonbrief.org/qa-how-integrated-assessment-models-are-used-to-study-climate-change/). IAMs are themselves fed assumptions about technological progress, wealth discrepancies, levels of international cooperation, and the like, and then spit out trajectories of population growth and gross domestic product (GDP) change consistent with RCPs. 

These narratives are called [Shared Socioeconomic Pathways](https://doi.org/10.1007/s10584-013-0905-2). There are five of them, each of which posits the possibilities for differing challenges to adaptation (i.e., managing climate impacts) and differing challenges to mitigation (i.e., preventing climate impacts). 

<center> <img src='https://climatedata.ca/site/assets/uploads/2022/03/ssp-graphics-01-1.png' width="900"></center>

These scenarios or pathways explore various hypothetical ways in which the world might change in the future, and so scientists combine the SSPs and RCPs together, and they are the **scenario** given to an Earth system model. They represent boundary conditions to a climate experiment, where the answer is a **projection** conditioned on the assumptions of the **scenario**. These are all used as part of [ScenarioMIP](https://gmd.copernicus.org/articles/9/3461/2016/), one of the coordinated experiments under [CMIP](https://www.wcrp-climate.org/wgcm-cmip). 

**Again, these scenarios have no likelihoods attached to them. They are not predictions of what will happen. They are a small sampling of scenario space -- a small set from the larger set of plausible futures. They are intended to give us a starting point to assess how climate change could unfold IF society looked one way versus another.**

Conveniently, FaIR has these emissions scenarios built-in for ease of simulation. 

With this background, let’s run FaIR’s built-in SSP scenarios and analyze their results below. 

We've coded this for you, but you should note the differences between this and our FaIR practical. 

In [ ]:
## We have written the code to do this for you. ##
## NOTE: This may take up to a minute to run because we are now computing multiple scenarios. ##

# init fair
f = FAIR(ch4_method='thornhill2021') # we want to enable the methane lifetime routine that is a function of SLCFs and reactive gases, see documentation

# set time horizon
# create world running from 1750 to 2100, at 1-year intervals
f.define_time(1750, 2100, 1)

# Define SSP scenarios (these all come from RCMIP)
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585'] 
f.define_scenarios(scenarios)

# define configs - still just the one default
f.define_configs(['default'])

# define species and properties
species, properties = read_properties()
f.define_species(species, properties)

# allocate for input and output data
f.allocate()

# fill in the data 

# (get default species configs)
f.fill_species_configs()

# get emissions + solar and volcanic forcing from RCMIP datasets using 'fill_from_rcmip()' helper function
f.fill_from_rcmip()

# initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

# fill climate configs using default middle-of-the-road vals from above
fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9])
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100])
fill(f.climate_configs["deep_ocean_efficacy"], 1.1)

# run FaIR
f.run(progress=False)

The results of our FaIR run are saved to this `f` object. 

-`f.temperature` is the `xarray DataArray` that stores temperature values for each layer, scenario, configuration, and time. 

-`f.timebounds` refers to the timesteps (years) used in the simulation. 

In order to access temperatures for specific times, configs, or scenarios, we would use the syntax `f.temperature.sel()` where the things we pass into the `sel()` method are our selection criteria. 

___________________________

### 1.1 First, let’s plot the time evolution of temperature for each SSP on a single time-temperature plot, where time is the x-axis and temperature is on the y-axis. 

Adjust the plotting code from the bonus section of the practical to make this plot. 

Below are the features the plot should include:

- i. Lines of temp anomaly vs. time for each SSP (from 1980-2100) 

- ii. SSP lines are colored by end-of-century warming (make the scenarios with higher temperature changes associated with warmer colors)

- iii. A legend that indicates which line corresponds to which SSP

- iv. A vertical dashed line at the present year (2023)

- v. Use the `ax.text()` method to add text saying ‘historical’ and ‘future’ on either side of the “present day” line

- vi. **BONUS**: Figure out how to plot over the 'historical' line in black, so that the plot looks more like one of the SSP plots, such as in the [Carbon Brief](https://www.carbonbrief.org/explainer-how-shared-socioeconomic-pathways-explore-future-climate-change/) article.

*HINT: This may seem like loads of work, but >95% of the code to do the above tasks are visible and clearly noted in the practical! There are many ways to achieve the same result here.*

______________
### 1.2 Interpret your results above. What are we looking at? Based on the references and links above, what is different about an SSP5-8.5 world vs. an SSP1-1.9 or SSP2-4.5 world? What SSPs are consistent with the IPCC recommendation to limit warming to less than 2$^{\circ}$C?
_______________

______________________
### 1.3 Some might consider these SSP experiments to be a sensitivity analysis. In this case, what is the scope of our sensitivity analysis? In other words, what are we allowing to vary (as proxied by different CO$_2$ pathways), and what "response" are we trying to study?
__________________________________

__________________________________
### 1.4 Are these experiments "transient" or "equilibrium" experiments? How do you know?
__________________________________

__________________________________
### 1.5 Now, let’s narrow in on two specific scenarios: SSP4-3.4 and SSP5-3.4. Replicate the plot above, plotting just those two scenarios. *As a* ***bonus***, *try to add a dashed, red, horizontal line at a temp anomaly of 2K.*
__________________________________

__________________________________
### 1.6 Given what you know about the SSP naming conventions, what are the main differences between these two scenarios? What real-world possibilities do they represent? Are those differences evident in the temp. vs. time graphs?
__________________________________

__________________________________
### 1.7 Let’s assume that every year above 2$^\circ$C of temperature anomaly has cascading harmful effects on human health, ecosystems, and the global economy. Using the temperature output for the 2 scenarios, compute the difference in years above 2$^\circ$C for SSP4-3.4 vs SSP5-3.4.
__________________________________

__________________________________
### 1.8 Based on your results above, does our emissions "pathway" matter for future temperatures, as well as other things we care about? Why or why not?
__________________________________

---
# 2. Climate Model "Ensembles" 
---

Just like the IPCC cannot rely on one scenario of future, so too can it not rely on one model to make an assessment. 

Instead, they rely on **ensembles,** or groups of climate models running the same experiment or scenario.

Using a handful of Earth System Models rather than just one is helpful because it allows one to see the potential range of responses to warming given uncertainty in the representation and parameterization of processes within the Earth’s climate - what scientists call **response** or **structural uncertainty**. 

Though FaIR is many many times simpler than a CMIP6 Earth System Model, the FaIR website has provided us information for how to manipulate FaIR’s climate parameters (that we briefly introduced in class) in order to “emulate” or replicate the results of 66 CMIP6 models. 

Below we will run FaIR again for a selected handful of SSPs and ***N configurations to emulate some certain number/handful of CMIP6 models.*** 


In [ ]:
## We have written the code to do this for you. ##
## NOTE: This may take up to 2 minutes to run because we are now computing multiple scenarios AND configurations. ##

# init fair
f = FAIR(ch4_method='thornhill2021') # we want to enable the methane lifetime routine that is a function of SLCFs and reactive gases, see documentation

# set time horizon
# create world running from 1750 to 2100, at 1-year intervals
f.define_time(1750, 2100, 1)

# Define SSP scenarios (these all come from RCMIP) (only grabbing a 3 representative SSPs this time)
scenarios = ['ssp119', 'ssp245', 'ssp585'] 
f.define_scenarios(scenarios)

# Define configs
# Our list of configs are going to be each CMIP6 climate model’s 4xCO2 response, which has been pre-calculated in the calibration notebooks.
# We could also modify the response for different aerosol, ozone, methane lifetime tunings etc., but not every model has these data available.
df = pd.read_csv('/home/jovyan/shared/GEOG60/Problem Sets/4xCO2_cummins_ebm3.csv') # the model data is stored in a csv that can be found on the FaIR github
models = df['model'].unique()
configs = []

for imodel, model in enumerate(models[:15]): # just going to use the first 15 models on file for the sake of this example
    for run in df.loc[df['model']==model, 'run']:
        configs.append(f"{model}_{run}")
f.define_configs(configs)


# define species and properties
species, properties = read_properties()
f.define_species(species, properties)

# allocate for input and output data
f.allocate()

# fill in the data 

# (get default species configs)
f.fill_species_configs()

# get emissions + solar and volcanic forcing from RCMIP datasets using 'fill_from_rcmip()' helper function
f.fill_from_rcmip()

# initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

# fill climate configs
# Take pre-calculated values from the Cummins et al. three layer model. We will use a reproducible random seed to define the stochastic behaviour. (per FaIR website)
seed = 1355763

for config in configs:
    model, run = config.split('_')
    condition = (df['model']==model) & (df['run']==run)
    fill(f.climate_configs['ocean_heat_capacity'], df.loc[condition, 'C1':'C3'].values.squeeze(), config=config)
    fill(f.climate_configs['ocean_heat_transfer'], df.loc[condition, 'kappa1':'kappa3'].values.squeeze(), config=config)
    fill(f.climate_configs['deep_ocean_efficacy'], df.loc[condition, 'epsilon'].values[0], config=config)
    fill(f.climate_configs['gamma_autocorrelation'], df.loc[condition, 'gamma'].values[0], config=config)
    fill(f.climate_configs['sigma_eta'], df.loc[condition, 'sigma_eta'].values[0], config=config)
    fill(f.climate_configs['sigma_xi'], df.loc[condition, 'sigma_xi'].values[0], config=config)
    fill(f.climate_configs['stochastic_run'], True, config=config)
    fill(f.climate_configs['use_seed'], True, config=config)
    fill(f.climate_configs['seed'], seed, config=config)

    seed = seed + 399

# run FaIR
f.run(progress=False)

Now, if we look at `f.temperature`, we'll see that we have 3 scenarios and 15 configs, where each config is an emulation of a different Earth system model!

__________________________________
### 2.1 Now we can make the same SSP plot as above, but include ranges of uncertainty that emerge based on the use of different models. Recreate the SSP plots you made before, but now include the **mean** and **range** across the CMIP6 ensemble for each SSP. 

- HINT: You could follow these steps. For each of our 4 SSPs:
    - Select the temperature data for that scenario and from 2023-2100 using your knowledge of xarray and `f.temperature.sel()`. Save into some var like `ssp_sel`. 
    - Generate ensemble min, max, and mean lines by reducing along the `config` dimension of your selected data. Refer to the Xarray Tutorial (and section "Reductions") for more support.
    - Use `ax.fill_between()` to fill the space between the ensemble range (min and max model values) for each SSP. Use the argument `alpha=0.2` for this function so that the ranges are partially transparent (there will help us visualize the model uncertainty ranges in the case of any overlap).
    - Use `ax.plot()` to plot your ensemble mean line for each SSP.
    - For both of those lines of plotting code, your x data will be your `ssp_sel.timebounds`, and your y data will be the min/max/mean data. 
    - Note: you may want to use the `zorder` plotting argument to ensure that your ensemble mean lines appear over the top of your model ranges! Refer to the plotting tutorial for more info.
- REMEMBER: Each model is designated in the dataset’s “config” dimension, while the SSPs are stored in the “scenario” dimension.

We have provided a code block with the outline for how you could do this plotting below. Recall there is the Xarry and plotting tutorials and the FaIR practical.
__________________________________

__________________________________
### 2.2 Typically, we consider the **ensemble mean** for some scenario to be the models’ “best guess” at the response to some forcing. However, some may argue that just considering the ensemble mean may not give us the full or accurate picture. Why might that be?
__________________________________

---
# 3. Uncertainty Partitioning
---

There are [three major sources of uncertainty](https://journals.ametsoc.org/view/journals/bams/90/8/2009bams2607_1.xml) climate scientists face when evaluating and interpreting climate model output:
- Scenario uncertainty, which is quantified as the range in answers from one climate model given different climate scenarios defined by the SSPs.
- Response, or structural uncertainty, which is quantified as the range of answers multiple models give in the same scenario.
- Internal variability uncertainty, which is quantified as the range of answers one model in one scenario can give.

Our FaIR dataset has two of the three uncertainties in it now. We sample multiple scenarions, so it has that uncertainty, and we sample multiple climate models (`configs`) so it has that uncertainty too.

We can try to do the same using our simplified FaIR experimental output. 

__________________________________
### 3.1 Quantify the range of variability (in K) due to model uncertainty vs. scenario uncertainty in our experiments. 
- *Hint: compute and compare the average temperature range within each SSP scenario at 2100 vs. the variation across all the entire range of SSPs at 2100. There are very many ways you could implement this using your knowledge of xarray, selection, and dimension reductions.*
__________________________________

__________________________________
### 3.2. What type of uncertainty is most important in this case? What does this tell you about decision making in the face of climate uncertainty and / or priorities for further climate model experimentation?
__________________________________

### CONGRATS! You're done. 
Document here who you worked with and about how long you spent on this, after reading [this article on task switching](https://www.nytimes.com/2023/01/09/well/mind/concentration-focus-distraction.html). Thanks!